In [13]:
from yelpapi import YelpAPI
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import time
import re

# camerons api key
#API_KEY = 'KrlCoAsmMTGJftfD6uV-s7kSN0o-SG2anMcTrwP6G4szLC3AsKX-hbiVLqiBfMj371sNemBCzrN8zCHae5SgIp1jqoiExhvExitIroX4_mBzQgWVtxcXFdFQIkkNYnYx'

# amys api key
#API_KEY = '3HutAvbl1TRuIXG0mjgNjTGPuLy_ZhoON7UEYzJqG6o2D6gPkIVZr1RT4x3TBpR3iXtJ9lBnis50H7MXRp0uSisapP-8Tup98qEipjCL6AjzLv2jHM6UhMXZN8IBYnYx'

#amys second api key
API_KEY = '_I5gcURUlBXeVjC_BsrLnA3yckqV4mF7O3bu4w6ZP0Dg8b0ooWNVw2QUujMftk6WcgR7g4CCxY36Z01ma6R3-yA6rYEkhl7p-T7tVC043sCihusZ1lvM3ZOU3fE4YnYx'

yelp_api = YelpAPI(API_KEY)

In [14]:
# STEP 1: create list of neighborhoods for each midwest city
cities = ['Chicago, Illinois',
 'Columbus, Ohio', 
 'Indianapolis, Indiana',
 'Detroit, Michigan',
 'Milwaukee, Wisconsin',
 'Kansas City, Missouri',
 'St. Louis, Missouri',
 'Minneapolis, Minnesota',
 'Saint Paul, Minnesota']    # changed St. Paul to Saint Paul

# scraping wikipedia page
# https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests
page = ''
while page == '':
    try:
        page = requests.get('https://en.wikipedia.org/wiki/Lists_of_neighborhoods_by_city#United_States')
        break
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        continue
        
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
items = content.find_all('li')

In [15]:
# STEP 1.1: get directories (what comes after wikipedia base url) for all cities
# creating dict where keys are cities and values are directories
city_directories = {}

# iterate through list of cities on wikipedia page - if they match our cities, get its directory
for item in items:
    wiki_city_cleaned = item.text.strip('*').strip()
    if wiki_city_cleaned == 'Neighborhoods of Chicago':   # hard-coded
        city_directories[cities[0]] = item.a['href']
    for city in cities:
        if wiki_city_cleaned == city.split(',')[0] or wiki_city_cleaned == city: 
            city_directories[city] = item.a['href']

city_directories['Kansas City, Missouri'] = '/wiki/Neighborhoods_of_Kansas_City,_Missouri'   # hard-coded
city_directories

{'Chicago, Illinois': '/wiki/Neighborhoods_of_Chicago',
 'Columbus, Ohio': '/wiki/Neighborhoods_in_Columbus,_Ohio',
 'Detroit, Michigan': '/wiki/Neighborhoods_in_Detroit',
 'Indianapolis, Indiana': '/wiki/Neighborhoods_of_Indianapolis',
 'Milwaukee, Wisconsin': '/wiki/Neighborhoods_of_Milwaukee',
 'Minneapolis, Minnesota': '/wiki/Neighborhoods_of_Minneapolis',
 'Saint Paul, Minnesota': '/wiki/Neighborhoods_of_Saint_Paul',
 'St. Louis, Missouri': '/wiki/Neighborhoods_of_St._Louis',
 'Kansas City, Missouri': '/wiki/Neighborhoods_of_Kansas_City,_Missouri'}

In [16]:
# STEP 1.2: creating functions for scraping wikipedia page & cleaning
base_url = 'https://en.wikipedia.org'

# uses pandas read_html to scrape neighborhoods from table on each wiki page
def read_wiki(city):
    directory = city_directories.get(city)
    wiki = pd.read_html(base_url + directory)
    return wiki

# uses bs4 to scrape neighborhoods based on html tags
def soup(city):
    directory = city_directories.get(city)
    page = requests.get(base_url + directory)
    soup = BeautifulSoup(page.text, 'html.parser')
    content = soup.find('div', class_='mw-parser-output')
    return content

# cleans where neighborhoods have multiple names with '/' and removes '[]'
def cleaned(lst):
    for neighborhood in lst:
        neighborhood = str(neighborhood)
        if '/' in neighborhood or ' / ' in neighborhood:
            lst.remove(neighborhood)
            aliases = neighborhood.split('/')
            for item in aliases:
                lst.append(item.strip())
        if '[' in neighborhood and neighborhood in lst:
            lst.remove(neighborhood)
            split = neighborhood.split('[')
            lst.append(split[0].strip())
    return lst


# https://stackoverflow.com/questions/8595973/truncate-to-three-decimals-in-python
import math
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

In [17]:
# STEP 1.3: get neighborhoods for chicago, indianapolis, detroit, st. louis
# creating dict where key is city and value is list of neighborhoods
neighborhoods = {}

# scraping chicago neighborhoods
wiki_chicago = read_wiki('Chicago, Illinois')
chicago_neighborhoods = wiki_chicago[0]
chicago_neighborhoods = list(chicago_neighborhoods['Neighborhood'])
neighborhoods['Chicago, Illinois'] = cleaned(chicago_neighborhoods)

# cleaning chicago neighborhoods

# # Replace 'Lakeview (East)' with Lake View East
# # Replace Legends South (Robert Taylor Homes) with Legends South, Robert Taylor Homes
# # Replace 'Lakeview (West)' with Lake View West
# neighborhoods['Chicago, Illinois'].replace('Lakeview (East)', 'Lakeview East')
# # neighborhoods['Chicago, Illinois'] = neighborhoods['Chicago, Illinois'].replace('Lakeview (West)', 'Lakeview West')
# # neighborhoods['Chicago, Illinois'] = neighborhoods['Chicago, Illinois'].replace('Legends South (Robert Taylor Homes)', 'Legends South')
# # neighborhoods['Chicago, Illinois']

# scraping indianapolis neighborhoods
wiki_ind = read_wiki('Indianapolis, Indiana')
ind_neighborhoods = wiki_ind[1]
indianapolis_neighborhoods = list(ind_neighborhoods[ind_neighborhoods.columns[0]])
neighborhoods['Indianapolis, Indiana'] = cleaned(indianapolis_neighborhoods)

# scraping st. louis neighborhoods
wiki_stl = read_wiki('St. Louis, Missouri')
stl_neighborhoods = wiki_stl[1]
stl_neighborhoods = list(stl_neighborhoods['Neighborhood'])
neighborhoods['St. Louis, Missouri'] = cleaned(stl_neighborhoods)

# scraping detroit neighborhoods
wiki_detroit =  read_wiki('Detroit, Michigan')
detroit_neighborhoods = []
for page in wiki_detroit:
    if 'Name' in page.columns:
        detroit_neighborhoods.extend(list(page['Name']))
        
# cleaning detroit neighborhoods
detroit_neighborhoods = cleaned(detroit_neighborhoods)
for n in detroit_neighborhoods:
    if '/' in n:
        detroit_neighborhoods.remove(n)
        names = n.split('/')
        for name in names:
            detroit_neighborhoods.append(name)
neighborhoods['Detroit, Michigan'] = detroit_neighborhoods

In [18]:
# STEP 1.4: get neighborhoods for columbus, milwaukee, kansas city, minneapolis, saint paul 
# NOTE: need fix repetition of code later for efficiency

# scraping columbus neighborhoods
columbus_directory = city_directories.get('Columbus, Ohio')
page = requests.get(base_url + columbus_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
columbus_neighborhoods = [item.text.replace('[edit]', '') for item in content.find_all(['h2', 'h3', 'h4'])][2:-3]
neighborhoods['Columbus, Ohio'] = cleaned(columbus_neighborhoods)

# scraping milwaukee neighborhoods
mlwk_directory = city_directories.get('Milwaukee, Wisconsin')
page = requests.get(base_url + mlwk_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
mlwk_neighborhoods = [item.text.replace('[edit]', '').strip('"') for item in content.find_all('h3')]
neighborhoods['Milwaukee, Wisconsin'] = cleaned(mlwk_neighborhoods)

# scraping kansas neighborhoods
kansas_directory = city_directories.get('Kansas City, Missouri')
page = requests.get(base_url + kansas_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
li_list = [item.text for item in content.find_all('li')]
kansas_neighborhoods = li_list[li_list.index('CBD-Downtown'):li_list.index('Wornall Homestead') + 1]
neighborhoods['Kansas City, Missouri'] = cleaned(kansas_neighborhoods)

# scraping minneapolis neightborhoods
minneapolis_directory = city_directories.get('Minneapolis, Minnesota')
page = requests.get(base_url + minneapolis_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3 = [item.text.replace('[edit]', '').strip('"') for item in content.find_all(['h3'])]
li = [item.text for item in content.find_all(['li']) if item.text[0].isalpha()]
li = li[0:li.index('University') + 1]
b = [item.text for item in content.find_all('b') if item.text[0].isalpha() and item.text not in li]
minneapolis_neighborhoods = h3 + li + b
neighborhoods['Minneapolis, Minnesota'] = cleaned(minneapolis_neighborhoods)

# scraping saint paul neighborhoods
stpaul_directory = city_directories.get('Saint Paul, Minnesota')
page = requests.get(base_url + stpaul_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3_list = [item.text.strip().split('- ')[1].replace('[edit]', '') for item in content.find_all('h3')]
stpaul_neighborhoods = []
for h3 in h3_list:
    names = h3.split(',')
    for name in names:
        stpaul_neighborhoods.append(name.strip())
neighborhoods['Saint Paul, Minnesota'] = cleaned(stpaul_neighborhoods)

In [19]:
# list of chains/fast food restaurants to remove
chains = pd.read_csv('cleaning/detroit_chains.csv')
dtw_chains = pd.read_csv('cleaning/detroit_local_chains.csv')
chains_names = list(chains['name'])
dtw_chains_names = list(dtw_chains['name'])

In [20]:
# STEP 2.1: get black owned restaurant id list for Detroit - used to create a column in csv file with Yelp API

# def get_black_owned(city):
#     black_owned = []

#     url = "https://api.yelp.com/v3/businesses/search"
#     headers = {'Authorization': f'Bearer {API_KEY}', 'user-agent': 'Mozilla/5.0'}

#     for n in neighborhoods[city]:
#         params = {
#             "category": "restaurants",
#             "location": city,
#             "attributes": "is_black_owned"
#         }
#         try:
#             response = requests.get(url, params=params, headers=headers).json()['businesses']
#             for business in response:
#                 if business['id'] not in black_owned:
#                     black_owned.append((business['id'], business['name']))
#                 else:
#                     continue
#         except KeyError:
#             continue
#     return black_owned

In [21]:
# result = get_black_owned('Detroit, Michigan')
# for tup in result:
#     print(tup)

In [22]:
# print(len(result))

In [23]:
def get_black_owned(city):
    city_name = city.split(',')[0]
    black_owned = []
    offset = 0
    for n in neighborhoods[city]:
        try:
            while offset <= 950:
                response = yelp_api.search_query(categories='restaurants', location=f"{n},{city_name}", attributes="is_black_owned", limit=50, offset=offset)['businesses']
                for business in response:
                    if business['id'] not in black_owned:
                        black_owned.append(business['id'])
                    else:
                        continue
                offset += 50
        except:
            continue
    return black_owned

In [24]:
cities = ['Kansas City, Missouri',
 'St. Louis, Missouri',
 'Minneapolis, Minnesota',
 'Saint Paul, Minnesota'] 

In [25]:
# STEP 2.3: use yelp API to get restaurant data for each neighborhood and write results into csv file
for city in cities:
    previous_restaurant_ids = []
    city_name = city.lower().split(',')[0]
    city_file_name = '_'.join(city_name.split()).replace('.', '')
    black_owned = get_black_owned(city)
    with open(f'{city_file_name}_restaurants.csv', "w", newline="") as fileout:
        header = fileout.write("id,name,review_count,rating,latitude,latitude_truncated,longitude,longitude_truncated,price,zip_code,is_black_owned\n") # add headers
        writer = csv.writer(fileout)
        count = 0
        for n in neighborhoods[city]:
            offset = 0
            try:
                # obtaining up to 1000 restaurants total restaurants
                while offset <= 1000:
                    result = yelp_api.search_query(categories='restaurants', location=n + f', {city_name}', limit=50, offset=offset)['businesses']
                    # parsing through json
                    current_idx = 0
                    for business in result:
                        current_idx += 1
                        restaurant_id = business['id']
                        name = business['name']
                        review_count = business['review_count']
                        rating = business['rating']
                        zip_code = business['location']['zip_code']
                        try:
                            latitude_truncated = str(truncate(business['coordinates']['latitude'], 5))
                            longitude_truncated = str(truncate(business['coordinates']['longitude'], 5))
                            latitude = str(business['coordinates']['latitude'])
                            longitude = str(business['coordinates']['longitude'])
                        except TypeError:
                            latitude = None
                            longitude = None
                        try:
                            price = business['price']
                        except KeyError:
                            price = None
                        if restaurant_id in previous_restaurant_ids or name in chains_names:
                            print('Skipping duplicates,fast food,chains...')
                            offset += current_idx
                            continue
                        else:
                            if restaurant_id in black_owned:
                                is_black_owned = 1
                            else:
                                is_black_owned = 0
                            row = restaurant_id,name,review_count,rating,latitude,latitude_truncated,longitude,longitude_truncated,price,zip_code,is_black_owned
                            writer.writerow(row)
                            count += 1
                            print(name + ' - '+ str(count))
                            previous_restaurant_ids.append(restaurant_id)                  
                    offset += 50    
            # way to handle errors if neighborhood does not give search results with Yelp API
            except YelpAPI.YelpAPIError as e:
                print('Error')
                continue
        print('Done')

Jack Stack Barbecue - Freight House - 1
Messenger Coffee - 2
Grünauer - 3
Novel Restaurant - 4
Q39 Midtown - 5
Cafe Gratitude - 6
Vietnam Cafe - 7
Tom's Town Distilling Co - 8
Tannin Wine Bar & Kitchen - 9
Bristol Seafood + Steak + Social - 10
BRGR kitchen + bar - 11
Extra Virgin - 12
The Westside Local - 13
Parlor - 14
Pigwich - 15
Happy Gillis Cafe & Hangout - 16
Mildred's - 17
Baramee Thai Bistro - 18
La Bodega - 19
The Rockhill Grille - 20
The Farmhouse - 21
Chez Elle Creperie & Coffeehouse - 22
801 Chophouse - 23
Banksia - 24
Grinders - 25
Affare - 26
Vested Garment District Coffee - 27
Town Topic - 28
Le Fou Frog - 29
Cosentino's Downtown Market - 30
County Road Ice House - 31
The Brick - 32
Joe's Kansas City Bar-B-Que - 33
Slap's BBQ - 34
Garozzo's Ristorante - 35
Blvd Tavern - 36
Succotash - 37
Los Alamos Market y Cocina - 38
Boulevard Brewing Tours & Rec Center - 39
Made In KC Cafe - 40
Kansas City Taco Company - 41
Farina - 42
Arthur Bryant's Barbeque - 43
Habashi House - 44


Bistro 303 - 349
Taqueria Los Altos de Jalisco - 350
First Watch - 351
Society - 352
Suzy's Deli - 353
Mad Jack's Fresh Fish - 354
Pizza 51 - 355
Banksia - 356
The Peanut - 357
China Star - 358
The Easy Inn - 359
Kin Lin Chinese Restaurant - 360
Sail Away Wine - 361
Tarahumara Mexican Restaurant - 362
Tacos On Wheels - 363
Restaurante El Pulgarcito - 364
Skipping duplicates,fast food,chains...
P.F. Chang's - 365
Nick's Italian Pizza - 366
Brian's Bakery & Eatery - 367
Tacos El Guero - 368
99 Hops House - 369
El Canelo Mexican Restaurant - 370
Third Street Social - South Plaza - 371
Planet Sub - 372
Prime Social - 373
Blu Hwy - 374
Big T's - 375
El Güero - 376
Paul & Jack's Tavern - 377
KC Burgers - 378
Burritos To Go Y Algo Mas - 379
Reggae Kitchen - 380
Lunch Box East Bottoms - 381
Rockhill Pizza - 382
Fritz's Railroad Restaurant - 383
Cooter Brand Food Truck - 384
El Camino Real - 385
Sankofa Cafe - 386
GreenAcres Market - Kansas City - 387
Chick-fil-A - 388
Duck and Roll - 389
Estre

Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...


Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...


Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...


...

Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...


Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...


Error
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Pho Pasteur VN - 371
Skipping duplicates,fast food,chains...
Turf Club - 372
Skipping duplicates,fast food,chains...
Burning Brothers Brewing - 373
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Checker Board Pizza - 374
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Hi-Lo Diner - 375
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Sh

Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Sakana Sushi & Asian Bistro - 463
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Agelgil Ethiopian Restaurant - 464
Hazelwood Food and Drinks - 465
Skipping duplicates,fast food,chains...
Buca di Beppo Italian Restaurant - 466
Skipping duplicates,fast food,chains...
Manny's Steakhouse - 467
Skipping duplicates,fast food,chain

Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
Skipping duplicates,fast food,chains...
